In [36]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

In [37]:
train = pd.read_csv("https://raw.githubusercontent.com/AMOGHA1140/kaggle_contents/main/titanic/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/AMOGHA1140/kaggle_contents/main/titanic/test.csv")


In [38]:
train['Age'].isnull().sum()

177

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [39]:
def preprocess(data):

  #remove useless data
  try:
    data.drop(["Name", "Ticket", "Cabin"], axis=1, inplace=True)
  except:
    None

  #one-hot-encode Sex and Embarked
  # enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
  encoded_data = pd.get_dummies(data[['Sex', 'Embarked']])
  data = pd.concat([data, encoded_data], axis=1)
  data.drop(['Sex', 'Embarked'], axis=1, inplace=True)
  # data['Embarked'] = enc.fit_transform(data[['Embarked']])

  #scale
  data['Age'].fillna(data['Age'].median(), inplace=True)
  data['Fare'].fillna(data['Fare'].median(), inplace=True)

  scaler = StandardScaler()
  data['Age'] = scaler.fit_transform(data[['Age']])
  data['Fare'] = scaler.fit_transform(data[['Fare']])

  return data



train = preprocess(train)
# train.describe()
train.head()
# train.columns'

<ipython-input-39-ea28c8a80be6>:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(data['Age'].mean(), inplace=True)
<ipython-input-39-ea28c8a80be6>:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,-0.592481,1,0,-0.502445,False,True,False,False,True
1,2,1,1,0.638789,1,0,0.786845,True,False,True,False,False
2,3,1,3,-0.284663,0,0,-0.488854,True,False,False,False,True
3,4,1,1,0.407926,1,0,0.420730,True,False,False,False,True
4,5,0,3,0.407926,0,0,-0.486337,False,True,False,False,True


In [40]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(['Survived', 'PassengerId'], axis=1), train['Survived'], test_size=0.2, random_state=42)

In [41]:
model1 = LogisticRegression()
model1.fit(X_train, y_train)
predictions = model1.predict(X_test)

print(len(predictions))

179


In [42]:
print(f"Accuracy: {accuracy_score(y_test, predictions)}")
print(f"F1 Score: {f1_score(y_test, predictions)}")

Accuracy: 0.8100558659217877
F1 Score: 0.7638888888888888


In [43]:
test = preprocess(test)
test.head()

<ipython-input-39-ea28c8a80be6>:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(data['Age'].mean(), inplace=True)
<ipython-input-39-ea28c8a80be6>:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,3,0.334993,0,0,-0.498407,False,True,False,True,False
1,893,3,1.325530,1,0,-0.513274,True,False,False,False,True
2,894,2,2.514175,0,0,-0.465088,False,True,False,True,False
3,895,3,-0.259330,0,0,-0.483466,False,True,False,False,True
4,896,3,-0.655545,1,1,-0.418471,True,False,False,False,True


In [45]:
submission = model1.predict(test.drop('PassengerId', axis=1))

submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': submission})
submission.to_csv('submission.csv', index=False)
print(submission)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
